In [1]:
from matplotlib import pyplot as plt
from copy import deepcopy
from scipy import integrate
%run "../HyperParameterOpt/GenerateExperiments/res_experiment.py"

In [2]:
plt.rcParams["figure.figsize"] = [10, 10]

## Impact of Network Sparsity on the Effective Adjacency Matrix of a Trained Reservoir Computer

Reservoir computer dynamics are governed by two equations:

$ \frac{d}{dt}\mathbf{r}(t) = \gamma[\, -\mathbf{r}(t) + \text{tanh}\big(\mathbf{A}\mathbf{r}(t) + \sigma \mathbf{W}_{\text{in}}\mathbf{u}(t)\big) \,]$

$\frac{d}{dt}\hat{\mathbf{r}}(t) = \gamma[-\hat{\mathbf{r}}(t) + \text{tanh}\big(\mathbf{A}\hat{\mathbf{r}}(t) + \sigma \mathbf{W}_{\text{in}}\mathbf{W}_{\text{out}}\hat{\mathbf{r}}(t)\big)$

The first equation is used to train the reservoir. In this equation, the nodes of the reservoir interact according to an adjacency matrix $\mathbf{A}$ and are perturbed by a desired output signal $\mathbf{u}$ transformed by $W_{\text{in}}$. The $-\mathbf{r}(t)$ term introduces an individual node activation decay when $\gamma$ is positive. Overall this means that the nodes in the reservoir naturally decay, but are excited by the combination of $\mathbf{A} \mathbf{r}(t)$ and $W_{in} \mathbf{u}(t)$. The hyperbolic tangent function is used to threshold exitiations from node interactions and stimulation from the desired output signal by ensuring that they never leave the interval $(-1,1)$.

During training, the first equation is solved for some time interval $(0, T)$. The solution $\mathbf{r}(t)$ is then projected onto $\mathbf{u}(t)$ via a linear mapping $W_{\text{out}}$. This mapping is the mapping that minimizes $||\mathbf{u}(t) - W_{\text{out}} \mathbf{r}(t)||_2 + \alpha || W_{\text{out}} ||_2$ for some amount of regularization ($\alpha$).

After training, the reservoir computer is governed by the second equation. At this point it is no longer dependent on the desired output signal $\mathbf{u}(t)$. It becomes a stand-alone dynamical system. 

However, from the network science perspective the structure of the reservoir network changes drastically after training. Where previously, reservoir node interactions were governed by $A$ and then stimulated by $W_{in} \mathbf{u}$, they are now governed by $\mathbf{A}\hat{\mathbf{r}}(t) + \sigma \mathbf{W}_{\text{in}}\mathbf{W}_{\text{out}}\hat{\mathbf{r}}(t)$, making the effective adjacency matrix $\mathbf{A} + \sigma \mathbf{W}_{\text{in}}\mathbf{W}_{\text{out}}$.

A few limited experiments found that $ \mathbf{W}_{\text{in}}\mathbf{W}_{\text{out}} $ did not have a clear component structure. But further work is needed to investigate the structure of the effective adjacency matrix

### Constants

In [3]:
DIFF_EQ_PARAMS = {
                  "x0": [-20, 10, -.5],
                  "begin": 0,
                  "end": 85,
                  "timesteps": 85000,
                  "train_per": .889,
                  "solver": lorenz_equ,
                  "clip": 40
                 }

RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-6,
              "signal_dim": 3,
              "network": "random graph",

              "res_sz": 15,
              "activ_f": np.tanh,
              "connect_p": .4,
              "spect_rad": 2.0,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }

### Functions

In [5]:
def remove_percent_edges(A, p):
    return remove_edges(A, floor(p * np.sum(A != 0)))

def train_rc(topo, remove_p=0.0, spect_rad=2.0, topo_p=None, n=2500):
    prms = deepcopy(RES_PARAMS)
    diff_eq_params = deepcopy(DIFF_EQ_PARAMS)
    # Make rc
    A = remove_percent_edges(generate_adj(topo, topo_p, n=n), remove_p)
    
    prms["spect_rad"] = spect_rad
    rc = ResComp(A, **prms)
    # Generate random orbit
    diff_eq_params["x0"] = random_lorenz_x0()
    train_t, test_t, u = rc_solve_ode(diff_eq_params)
    err = rc.fit(train_t, u)
    pred = how_long_accurate(u(test_t), rc.predict(test_t), tol=TOL)
    return rc, err, pred

def effective_adj(rc):
    return rc.res + rc.sigma * rc.W_in @ rc.W_out
    
def plot_matrix(A, log=False, title="Effective Adj Matrix"):
    if log:
        A = np.log(np.abs(A))
        title += " (Pictured: log(A))"
    plt.imshow(A, cmap='hot')
    plt.colorbar()
    plt.title(title)
    plt.show()

In [10]:
def test_train_rc(topo, remove_p=0.0, spect_rad=2.0, topo_p=None, n=2500):
    prms = deepcopy(RES_PARAMS)
    diff_eq_params = deepcopy(DIFF_EQ_PARAMS)
    # Make rc
    A = remove_percent_edges(generate_adj(topo, topo_p, n=n), remove_p)
    
    prms["spect_rad"] = spect_rad
    rc = ResComp(A, **prms)
    # Generate random orbit
    diff_eq_params["x0"] = random_lorenz_x0()
    train_t, test_t, u = rc_solve_ode(diff_eq_params)
    err = rc.fit(train_t, u)
    pred = how_long_accurate(u(test_t), rc.predict(test_t), tol=TOL)
    return rc, pred, u, test_t

def predict(rc, t, eadj = effective_adj(rc), u_0=None, r_0=None, return_states=False):
    """
        Parameters:
        -----------
    """
    if not rc.is_trained:
        raise Exception("Reservoir is untrained")

    # Reservoir prediction ode
    def res_pred_f(r,t):
        return rc.gamma*(-1*r + rc.activ_f(eadj @ r))
    # end
    if r_0 is None and u_0 is None:
        r_0  = rc.state_0
    # end
    elif r_0 is None and u_0 is not None:
        r_0 = rc.W_in @ u_0
    # end
    pred = integrate.odeint(res_pred_f, r_0, t)
    if return_states:
        return rc.W_out @ pred.T, pred.T
    return rc.W_out @ pred.T

In [89]:
RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-6,
              "signal_dim": 3,
              "network": "random graph",
              #"mean_degree": 4,
              "res_sz": 15,
              "activ_f": np.tanh,
              "connect_p": .4,
              "spect_rad": 0.1,
              "gamma": 2,
              "sigma": .14,
              "sparse_res": True,
             }
rc, pred, u, test_t = test_train_rc("barab1",remove_p=.94)
print(pred)

1786


In [91]:
predictions_bara1 = [(pred,0)]
for thin in np.logspace(-5,-3,20):
    eadj = effective_adj(rc)
    percent = np.sum(np.abs(eadj)<=thin)/np.size(eadj)
    eadj = np.where(np.abs(eadj)>thin,eadj,0)
    pred1 = how_long_accurate(u(test_t), predict(rc,test_t,eadj), tol=TOL)
    predictions_bara1.append((pred1,percent))
print(predictions_bara1)

[(1786, 0), (1786, 3.088e-05), (1786, 4e-05), (1786, 5.216e-05), (1786, 6.752e-05), (1784, 8.832e-05), (1782, 0.00011136), (1777, 0.00014096), (1776, 0.00017936), (1780, 0.00022352), (1772, 0.00028352), (1788, 0.00035888), (1777, 0.00046064), (1802, 0.00058816), (1766, 0.00073584), (1136, 0.00093712), (1776, 0.00118256), (1754, 0.0014832), (1875, 0.0018408), (1247, 0.002248), (1917, 0.0027488)]


In [81]:
RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-6,
              "signal_dim": 3,
              "network": "random graph",
              #"mean_degree": 4,
              "res_sz": 15,
              "activ_f": np.tanh,
              "connect_p": .4,
              "spect_rad": 0.1,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }
rc, pred, u, test_t = test_train_rc("barab2",remove_p=.84)
print(pred)

788


In [82]:
predictions_bara2 = [(pred,0)]
for thin in np.logspace(-5,-3,20):
    eadj = effective_adj(rc)
    percent = np.sum(np.abs(eadj)<=thin)/np.size(eadj)
    eadj = np.where(np.abs(eadj)>thin,eadj,0)
    pred1 = how_long_accurate(u(test_t), predict(rc,test_t,eadj), tol=TOL)
    predictions.append((pred1,percent))
print(predictions_bara2)

[(788, 0), (788, 4.24e-05), (786, 5.36e-05), (788, 7.056e-05), (791, 9.104e-05), (789, 0.00011632), (789, 0.0001456), (792, 0.00018144), (796, 0.00022784), (810, 0.00029136), (849, 0.0003624), (1355, 0.00046128), (1208, 0.00058512), (1191, 0.00075024), (731, 0.0009648), (681, 0.00121664), (735, 0.00156992), (610, 0.00199744), (527, 0.0025544), (461, 0.0032536), (443, 0.00417728)]


In [95]:
RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-4,
              "signal_dim": 3,
              "network": "random graph",
              #"mean_degree": 4,
              "res_sz": 15,
              "activ_f": np.tanh,
              "connect_p": .4,
              "spect_rad": 1.,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }
rc, pred, u, test_t = test_train_rc("erdos",topo_p=2,remove_p=.96)
print(pred)

2529


In [96]:
predictions_erdos2 = [(pred,0)]
for thin in np.logspace(-5,-3,20):
    eadj = effective_adj(rc)
    percent = np.sum(np.abs(eadj)<=thin)/np.size(eadj)
    eadj = np.where(np.abs(eadj)>thin,eadj,0)
    pred1 = how_long_accurate(u(test_t), predict(rc,test_t,eadj), tol=TOL)
    predictions_erdos2.append((pred1,percent))
print(predictions_erdos2)

[(2529, 0), (2524, 0.00018016), (2522, 0.00022944), (2518, 0.00028672), (2520, 0.0003664), (2527, 0.0004648), (2510, 0.00058688), (2546, 0.00075216), (2495, 0.00095264), (2523, 0.0012104), (2472, 0.0015368), (4584, 0.00196704), (4995, 0.00250848), (4995, 0.00318192), (2417, 0.00405888), (2462, 0.00517952), (2483, 0.00657904), (2337, 0.00839616), (2277, 0.01075104), (2312, 0.01372464), (2184, 0.01745952)]


In [7]:
RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-8,
              "signal_dim": 3,
              "network": "random graph",
              #"mean_degree": 3,
              "res_sz": 15,
              "activ_f": np.tanh,
              "connect_p": .4,
              "spect_rad": 1.0,
              "gamma": 5.0,
              "sigma": .14,
              "sparse_res": True,
             }
rc, pred, u, test_t = test_train_rc("random_digraph",remove_p=.96,topo_p=2)
print(pred)

C:\Users\joebo\WebbWork\ReservoirSpecialization\rescomp\ResComp.py:167: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")
C:\Users\joebo\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.94251e-17): result may not be accurate.
  overwrite_a=True).T


703


In [11]:
predictions_random_digraph = [(pred,0)]
for thin in np.logspace(-5,-3,10):
    eadj = effective_adj(rc)
    percent = np.sum(np.abs(eadj)<=thin)/np.size(eadj)
    eadj = np.where(np.abs(eadj)>thin,eadj,0)
    pred1 = how_long_accurate(u(test_t), predict(rc,test_t,eadj), tol=TOL)
    predictions_random_digraph.append((pred1,percent))
print(predictions_random_digraph)

[(703, 0), (705, 3.84e-06), (687, 5.76e-06), (770, 9.6e-06), (620, 1.568e-05), (596, 2.496e-05), (599, 4.16e-05), (506, 6.752e-05), (466, 0.00011792), (474, 0.00019696), (448, 0.00032656)]


In [19]:
RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-8,
              "signal_dim": 3,
              "network": "random graph",
              #"mean_degree": 4,
              "res_sz": 15,
              "activ_f": np.tanh,
              "connect_p": .4,
              "spect_rad": 2.,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }
rc, pred, u, test_t = test_train_rc("watts2",topo_p=.5,remove_p=.98)
print(pred)

C:\Users\joebo\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=2.75022e-17): result may not be accurate.
  overwrite_a=True).T


2394


In [20]:
predictions_watts2 = [(pred,0)]
for thin in np.logspace(-5,-3,6):
    eadj = effective_adj(rc)
    percent = np.sum(np.abs(eadj)<=thin)/np.size(eadj)
    eadj = np.where(np.abs(eadj)>thin,eadj,0)
    pred1 = how_long_accurate(u(test_t), predict(rc,test_t,eadj), tol=TOL)
    predictions_watts2.append((pred1,percent))
print(predictions_watts2)

[(2394, 0), (2392, 2.56e-06), (2306, 8.32e-06), (2171, 2.336e-05), (1363, 5.248e-05), (706, 0.0001384), (496, 0.00034304)]


In [14]:
RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-6,
              "signal_dim": 3,
              "network": "random graph",
              #"mean_degree": 4,
              "res_sz": 15,
              "activ_f": np.tanh,
              "connect_p": .4,
              "spect_rad": 2.,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }
rc, pred, u, test_t = test_train_rc("watts4",topo_p=.1,remove_p=.99)
print(pred)

3342


In [16]:
predictions_watts4 = [(pred,0)]
for thin in np.logspace(-5,-3,20):
    eadj = effective_adj(rc)
    percent = np.sum(np.abs(eadj)<=thin)/np.size(eadj)
    eadj = np.where(np.abs(eadj)>thin,eadj,0)
    pred1 = how_long_accurate(u(test_t), predict(rc,test_t,eadj), tol=TOL)
    predictions_watts4.append((pred1,percent))
print(predictions_watts4)

[(3342, 0), (3348, 2.208e-05), (3335, 2.928e-05), (3323, 3.872e-05), (3320, 5.024e-05), (3343, 6.016e-05), (3291, 7.92e-05), (3314, 0.00010064), (3244, 0.00012544), (3494, 0.00016128), (3344, 0.0002048), (3293, 0.0002632), (1179, 0.00033056), (1183, 0.00042), (3279, 0.00053504), (1093, 0.00067968), (1048, 0.00087488), (1082, 0.00110944), (869, 0.00141664), (879, 0.00181296), (848, 0.0023176)]


Test if removing largest edges is effective

In [103]:
RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-6,
              "signal_dim": 3,
              "network": "random graph",
              #"mean_degree": 4,
              "res_sz": 15,
              "activ_f": np.tanh,
              "connect_p": .4,
              "spect_rad": 0.1,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }
rc, pred, u, test_t = test_train_rc("barab2",remove_p=.84)
print(pred)

907


In [124]:
predictions_bara = [(pred,0)]
for thin in np.linspace(3,np.max(effective_adj(rc)),9):
    eadj = effective_adj(rc)
    percent = np.sum(np.abs(eadj)>=thin)/np.size(eadj)
    eadj = np.where(np.abs(eadj)<thin,eadj,0)
    pred1 = how_long_accurate(u(test_t), predict(rc,test_t,eadj), tol=TOL)
    predictions_bara.append((pred1,percent))
print(predictions_bara)

[(907, 0), (3, 0.0096808), (2, 0.00291536), (1, 0.00109424), (3, 0.00049344), (4, 0.00024176), (3, 0.00012224), (3, 5.568e-05), (5, 1.136e-05), (50, 1.6e-07)]
